<a href="https://colab.research.google.com/github/Trrunghau/AI/blob/main/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# khai báo các thư viện cần sử dụng
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Activation
from keras.optimizers import Adam
from keras.layers import LeakyReLU
from keras.models import load_model
from os import listdir
from numpy import asarray,save
from keras.utils import load_img, img_to_array
import time
import cv2
import altair as alt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2,preprocess_input as mobilenet_v2_preprocess_input
from PIL import Image



In [17]:

#==============Cân bằng histogram===================================
def equal_hist(imgPIL):
     # ảnh này dùng để chứa kế quả chuyển đổi RGB sang Grayscale
    average = Image.new(imgPIL.mode,imgPIL.size)
    # lấy kích thước của ảnh từ imgPIL
    width  = average.size[0]
    height = average.size[1]
    # mỗi ảnh là một ma trận chiều
    for x in range(width):
        for y in range(height):
            # lấy giá trị điểm ảnh tại vị trí x,y
            R,G,B = imgPIL.getpixel((x,y))

            #Chuyển đổi điểm ảnh màu RGB sang mức xám dùng phương pháp Luminance
            grayLuminance= np.uint8(0.2126*R + 0.7152*G + 0.0722*B)

            # gán giá trị múc xám vừa tính cho ảnh xám
            average.putpixel((x,y),(grayLuminance,grayLuminance,grayLuminance))

    # mỗi pixel có giá trijw từ 0-255, nên khai báo mảng có 256 pt
    his = np.zeros(256)
    for x in range(width):
        for y in range(height):
            # lấy giá trị xám tại vị trí x,y
            gR, gG, gB = average.getpixel((x,y))

            #giá trị gray tính ra cũng chính là phần tử thứ gray
            # trong mảng his đã khai báo ở trên, tăng số đếm của phần tử thứ gray lên 1
            his[gR] += 1
    print('/n')
    cumulator = np.zeros_like(his, np.float64)
    for i in range(len(cumulator)):
        cumulator[i] = his[:i].sum()
    new_hist = (cumulator - cumulator.min())/(cumulator.max() - cumulator.min()) * 255
    new_hist = np.uint8(new_hist)
    new_Img = Image.new(img.mode,img.size)
    for i in range(height):
        for j in range(width):
            R,G,B = average.getpixel((i,j))
            new_Img.putpixel((i,j),(new_hist[R],new_hist[R],new_hist[R]))
    return new_Img


In [18]:
def changeImg(imgPIL):
    # tạo ảnh chứa kết quả chuyển đổi
    img = Image.new(imgPIL.mode,imgPIL.size)
    a = (3)/2
    a = int(a)
    # lấy kích thước của ảnh từ imgPIL
    width  = img.size[0]
    height = img.size[1]

    for x in range(a,width-a):
        for y in range(a,height-a):

            Rs = 0; Gs = 0; Bs = 0
            for i in range(x-a,x+a+1):
                for j in range(y-a,y+a+1):

                    # lấy giá trị điểm ảnh tại vị trí x,y
                    R,G,B = imgPIL.getpixel((i,j))
                    Rs = Rs + R
                    Gs = Gs + G
                    Bs = Bs + B
            K = 9
            Rs = float(Rs)/float(K)
            Gs = float(Gs)/float(K)
            Bs = float(Bs)/float(K)
            img.putpixel((x,y),(np.uint8(Bs),np.uint8(Gs),np.uint8(Rs)))
    return img

In [19]:

#tạo hàm làm sắc nét ảnh
k=[[0,1,0],[1,-4,1],[0,1,0]]
def Sacnet(imgPIL):
    sacnet = Image.new(imgPIL.mode, imgPIL.size)
    width = sacnet.size[0]
    height = sacnet.size[1]
    for x in range(1, width - 1):
        for y in range(1, height - 1):
            rs = 0
            gs = 0
            bs = 0
            rs1 = 0
            gs1 = 0
            bs1 = 0
            a = 0
            b = 0
            for i in range(x - 1, x + 1 + 1):
                for j in range(y - 1, y + 1 + 1):
                    color = imgPIL.getpixel((i, j))
                    R = color[0]
                    G = color[1]
                    B = color[2]
                    rs += R * k[a][b]
                    gs += G * k[a][b]
                    bs += B * k[a][b]
                    b += 1
                    if b == 3:
                        b = 0
                a += 1
                if a == 3:
                    a = 0
            R1, G1, B1 = imgPIL.getpixel((x, y))
            rs1 = R1 - rs
            gs1 = G1 - gs
            bs1 = B1 - bs

            if rs1 > 255:
                rs1 = 255
            elif rs1 < 0:
                rs1 = 0
            if gs1 > 255:
                gs1 = 255
            elif gs1 < 0:
                gs1 = 0
            if bs1 > 255:
                bs1 = 255
            elif bs1 < 0:
                bs1 = 0

            sacnet.putpixel((x, y), (bs1, gs1, rs1))
    return sacnet

In [27]:
folder = '/content/drive/MyDrive/TEST/NORMAL'
for file in listdir(folder) :
    img = '/content/drive/MyDrive/TEST/NORMAL/' + file
    img = image.load_img(img,target_size=(64,64))
    img = changeImg(img)
    img = equal_hist(img)
    img = Sacnet(img)
    
    img = image.array_to_img(img)
    img.save('/content/drive/MyDrive/test2/normal/'+file)

/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
/n
